In [1]:
import numpy as np
from sktime.classification.kernel_based import RocketClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
from sktime.datasets import load_arrow_head  # univariate dataset
X_train, y_train = load_arrow_head(split="test", return_X_y=True)
X_test, y_test = load_arrow_head(split="train", return_X_y=True)
print(X_train.shape, X_test.shape)

(175, 1) (36, 1)


In [3]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested,
    from_nested_to_2d_array,
    is_nested_dataframe,
)

x_np = from_nested_to_2d_array(X_train)
#x = x_np.iloc[:10, :10] # Using first 5 steps for faster training
X_train = from_2d_array_to_nested(x_np)


In [4]:
# ECG dataset
# from sktime.datasets import load_from_arff_to_dataframe
# from IPython.display import display

# X_train, y_train = load_from_arff_to_dataframe('./ECG5000/ECG5000_TRAIN.arff')
# X_test, y_test = load_from_arff_to_dataframe('./ECG5000/ECG5000_TEST.arff')


In [5]:
rocket = RocketClassifier()
rocket.fit(X_train, y_train)
y_pred = rocket.predict(X_test)
print(X_train)
accuracy_score(y_test, y_pred)

                                                     0
0    0     -1.907777
1     -1.904890
2     -1.88856...
1    0     -1.829855
1     -1.808497
2     -1.79577...
2    0     -1.801563
1     -1.775734
2     -1.72848...
3    0     -1.885718
1     -1.855170
2     -1.84519...
4    0     -1.959051
1     -1.974930
2     -1.97141...
..                                                 ...
170  0     -1.625142
1     -1.622988
2     -1.62606...
171  0     -1.657757
1     -1.664673
2     -1.63264...
172  0     -1.603279
1     -1.587365
2     -1.57740...
173  0     -1.739020
1     -1.741534
2     -1.73286...
174  0     -1.630727
1     -1.629918
2     -1.62055...

[175 rows x 1 columns]


0.8888888888888888

In [6]:

X_pyreal = np.empty((X_train.shape[0], X_train.iloc[0][0].shape[0]))
print(X_pyreal.shape)
for i in range(X_train.shape[0]):
    for j in range(X_train.iloc[0][0].shape[0]):
        X_pyreal[i,j] = X_train.iloc[i][0][j]

X_pyreal = pd.DataFrame(X_pyreal)

(175, 251)


In [7]:
from sktime.datatypes._panel._convert import (
    from_2d_array_to_nested
)
class wrapped_model(object):
    def __init__(self, model):
        self.model = model

    def __call__(self, X):
        self.predict(X)

    def transform(self, data):
        df = from_2d_array_to_nested(data)
        return df

    def predict(self, X):
        X = self.transform(X)
        # without astype, the output is in <u1
        return self.model.predict(X).astype('int')

model = wrapped_model(rocket)
print(model.predict(X_pyreal))
print(X_pyreal.shape)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
(175, 251)


In [8]:
from pyreal.explainers.time.occlusion_importance import OcclusionSaliency

# change window_size to aggregate different timesteps
explainer = OcclusionSaliency(rocket, X_pyreal, window_size=3, classes=['0', '1', '2'])
explainer.fit()
explanation = explainer.produce(X_pyreal.iloc[:1, :])

print(explanation)


[[-1.9077772  -1.9048903  -1.8885626  -1.8711639  -1.8316792  -1.8065202
  -1.7657007  -1.7158483  -1.6605015  -1.5988998  -1.5654527  -1.4927228
  -1.3858783  -1.3388661  -1.2612161  -1.1566997  -1.0565757  -0.97513866
  -0.84251433 -0.69873062 -0.56411964 -0.49159144 -0.34189434 -0.15826901
  -0.09750549  0.07021113  0.23266624  0.35825193  0.42631328  0.55423639
   0.65272451  0.70215947  0.75214697  0.82944934  0.91872593  0.97683541
   0.94865604  0.99289622  1.0683703   1.1135463   1.1798073   1.2224954
   1.2835942   1.3247395   1.4298022   1.5080467   1.5154345   1.5043078
   1.5451598   1.5626423   1.5626423   1.5802536   1.5595071   1.5177704
   1.5177704   1.4370504   1.4135179   1.3492389   1.3451166   1.318002
   1.288404    1.2635989   1.1879479   1.0860716   1.044352    1.0133494
   0.94901456  0.89966962  0.84737575  0.76695992  0.6976414   0.61879007
   0.55337344  0.44777685  0.36540387  0.27987398  0.25740933  0.17573633
   0.18829257  0.20011934  0.19631677  0.16494

invalid value encountered in true_divide


ValueError: not enough values to unpack (expected 2, got 1)